# Prediction Model 1 - with Boston Housing dataset

**[Tutorial]** 본격적으로 들어가기 전에 먼저 독립 변수가 한 개인 가장 기본적인 Linear Regression을 적합시켜 봅시다.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from sklearn.datasets import make_regression

plt.figure()
plt.title('Sample regression problem with one input variable')
X, y = make_regression(n_samples = 100, n_features=1,n_informative=1, 
                             bias = 150.0, noise = 30, random_state=1)
plt.scatter(X, y, marker= 'o', s=50)
plt.show()

위와 같이 random data를 생성했습니다. 이제 Linear Regression을 적용해봅시다.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)
reg = LinearRegression()
reg.fit(X_train, y_train)

print('linear model coef (w): {}'.format(reg.coef_))
print('linear model intercept (b): {:.3f}'.format(reg.intercept_))
print('R-squared score (training): {:.3f}'.format(reg.score(X_train, y_train)))
print('R-squared score (test): {:.3f}'.format(reg.score(X_test, y_test)))

적합시킨 모델을 그림 위에 나타내어 봅시다.

In [ ]:
plt.figure()
plt.scatter(X, y, marker= 'o', s=50, alpha=0.8)
plt.plot(X, reg.coef_ * X + reg.intercept_, 'r-')
plt.title('Least-squares linear regression')
plt.xlabel('Feature value (x)')
plt.ylabel('Target value (y)')
plt.show()

이제 본격적으로 **보스턴 하우징 데이터**를 이용하여 2주차 세션에서 배웠던 Linear, Ridge, Lasso regression, Elastic Net을 적용해봅시다. 

다음의 모듈을 import 하세요.

- sklearn.datasets : load_boston
- sklearn.linear_model : LinearRegression, Ridge, Lasso, ElasticNet
- sklearn.model_selection : cross_val_score

- load_boston 함수를 이용하여 boston이라는 이름으로 데이터를 로드하세요.
boston 데이터를 파악하세요.

In [ ]:
boston = ______
print(boston.DESCR)

In [ ]:
df=pd.DataFrame(boston.data,columns=boston.feature_names)
df['target']=boston.target
df.head()

In [ ]:
X=df.drop('target',axis=1)
y=df['target']

 **Linear Regression**

- train set과 test set을 분리하세요. 이 때, test set의 크기는 0.3, random state는 20으로 하세요.
- reg_linear라는 이름으로 Linear Regression을 생성하세요.
- train set에 모델을 적합시키세요.

In [ ]:
X_train, X_test, y_train, y_test = ______(__, __, ______=___, ______ = ___)
reg_linear =_______
______.___(______, ______)

cv_linear=cross_val_score(estimator=reg_linear, X=X_train, y=y_train, cv=10)

print("CV score: ",cv_linear.mean())
print('R-squared score (training): {:.3f}'.format(reg_linear.score(X_train, y_train)))
print('R-squared score (test): {:.3f}'.format(reg_linear.score(X_test, y_test)))

**Ridge Regression**

- alpha의 값을 0.005, normalize=True로 하여, reg_ridge라는 이름으로 Ridge Regression을 생성하세요.
- train set에 대하여 모델을 적합시키세요.

In [ ]:
reg_ridge=_____(____=____, ______=____)
______.___(______, ______)

cv_ridge=cross_val_score(reg_ridge, X_train, y_train, cv=10)

print("CV score: ",np.mean(cv_ridge))
print("R^2 for training data: {}".format(reg_ridge.score(X_train, y_train)))
print("R^2 for test data: {}".format(reg_ridge.score(X_test,y_test)))

알파의 값을 임의로 지정해서 적합시켜 보았습니다.
​
 그럼 이제 알파의 값에 따라 CV 스코어가 어떻게 변하는지 그림으로 나타내어 봅시다. 아래의 코드를 실행시켜 보세요.

In [ ]:
def display_plot(cv_scores, cv_scores_std):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(alpha_space, cv_scores)

    std_error = cv_scores_std / np.sqrt(10)

    ax.fill_between(alpha_space, cv_scores + std_error, cv_scores - std_error, alpha=0.2)
    ax.set_ylabel('CV Score +/- Std Error')
    ax.set_xlabel('Alpha')
    ax.axhline(np.max(cv_scores), linestyle='--', color='.5')
    ax.set_xlim([alpha_space[0], alpha_space[-1]])
    ax.set_xscale('log')
    plt.show()

In [ ]:
ridge = Ridge(normalize=True)

alpha_space = np.logspace(-6, 0, 50)
ridge_scores = []
ridge_scores_std = []

for alpha in alpha_space:
    ridge.alpha = alpha
    ridge_cv_scores = cross_val_score(ridge, X_train, y_train, cv=10)
    ridge_scores.append(np.mean(ridge_cv_scores))
    ridge_scores_std.append(np.std(ridge_cv_scores))
    
display_plot(ridge_scores, ridge_scores_std)

번외로 Ridge Regression에 Polynomial Feature를 추가한 모델을 적합시켜 보겠습니다. 

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

steps = [
    ('scalar', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2)),
    ('model', Ridge(alpha=2, fit_intercept=True))
]

ridge_pipe = Pipeline(steps)
ridge_pipe.fit(X_train, y_train)

cv_ridge = cross_val_score(estimator = ridge_pipe, X = X_train, y = y_train.ravel(), cv = 10)

print('CV: ', cv_ridge.mean())
print("R^2 for training data: {}".format(ridge_pipe.score(X_train, y_train)))
print("R^2 for test data: {}".format(ridge_pipe.score(X_test,y_test)))

**Lasso Regresson**
- alpha의 값을 0.005, normalize=True로 하여, reg_lasso라는 이름으로 Lasso Regression을 생성하세요.
- train set에 대하여 모델을 적합시키세요.
- 적합시킨 모델과 train set에 대하여 cross_val_score를 호출하세요. 이때 cv=10으로 하세요.

In [ ]:
reg_lasso = _____(____=_____, ______=____)
______.___(______, ______)

cv_lasso=_________(________,______, ______, ___=__)

print("CV score: ",cv_lasso.mean())
print("R^2 for training data: {}".format(reg_lasso.score(X_train, y_train)))
print("R^2 for test data: {}".format(reg_lasso.score(X_test,y_test)))

**Lasso**의 경우도 알파의 값에 따라 CV 스코어가 어떻게 변화하는지 그림으로 나타내어 봅시다.
위 Ridge Regression의 경우를 참고하세요.

- lasso라는 이름으로 Lasso Regression을 생성하세요. 이때, normalize=True로 하세요.
- train set에 대하여 cross_val_score 함수를 호출하세요. 이 때, cv=10으로 하세요.
- display_plot을 호출하세요.

In [ ]:
lasso = _____(______=____)

alpha_space = np.logspace(-6, 0, 50)
lasso_scores = []
lasso_scores_std = []

for alpha in alpha_space:
    lasso.alpha = alpha
    lasso_cv_scores = ________(_____, ______, ______, ___=__)
    lasso_scores.append(np.mean(lasso_cv_scores))
    lasso_scores_std.append(np.std(lasso_cv_scores))
    
______(________, ________)

마찬가지로, Lasso Regression에 Polynomial Feature를 추가하여 적합시켜 봅시다.

- 스케일러는 StandardScaler을 이용하세요.
- 모델은 알파=0.02로 하는 Lasso Regression을 이용하세요.
- Pipeline을 적용하세요.
- train data에 대하여 Pipeline을 적합시키세요.

In [ ]:
steps = [
    ('scalar', _________),
    ('poly', PolynomialFeatures(degree=2)),
    ('model',_____(____=____, fit_intercept=True))
]

lasso_pipe = ______(_____)
________.___(______, ______)

cv_lasso = cross_val_score(estimator = lasso_pipe, X = X_train, y = y_train, cv = 10)

print("CV score: ", cv_lasso.mean())
print("R^2 for training data: {}".format(lasso_pipe.score(X_train, y_train)))
print("R^2 for test data: {}".format(lasso_pipe.score(X_test,y_test)))

**Elastic Net**

마지막으로 Elastic Net을 간단하게 적용해보고 마무리합시다. Elastic Net을 적용시키기 위해서는 최적의 모수 조합을 찾아야 합니다. 

GirdSearchCV를 이용하여 최적의 모수 조합을 찾도록 합시다. 다음의 모듈을 import 하세요.

- sklearn.model_selection : GridSearchCV
- sklearn.metrics : mean_squared_error

- reg_enet이라는 이름으로 ElasticNet을 생성하세요. 이때, normalize=True로 하세요.
- train set에 대하여 search를 적합시키세요.

In [ ]:
reg_enet=_______(______=____)
search=GridSearchCV(estimator=reg_enet,param_grid={'alpha':np.logspace(-5,2,8),'l1_ratio':[.2,.4,.6,.8]},
                    scoring='neg_mean_squared_error',n_jobs=1,refit=True,cv=10)
______.____(______,______)
search.best_params_

- 위 결과에서 나온 알파와 ㅣ1_ratio의 값을 대입하여 ElasticNet을 생성하세요.
- train data에 대하여 모델을 적합시키세요. 

In [ ]:
reg_enet=_______(normalize=True, alpha=______, l1_ratio=_____)
_______.___(______, ______)

cv_lasso=cross_val_score(reg_enet, X_train, y_train, cv=10)

print("CV score: ",cv_lasso.mean())
print("R^2 for training data: {}".format(reg_enet.score(X_train, y_train)))
print("R^2 for test data: {}".format(reg_enet.score(X_test,y_test)))

수고하셨습니다.